In [1]:
import copy
import torch
import random
import pandas as pd

from itertools import product
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

random_state = 42
preproc = True

In [2]:
def fix_random(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(random_state)

## Device

In [3]:
# PyTorch Device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("Device: {}".format(device))

Device: cpu


## Data Loading


In [4]:
df = pd.read_csv("train.csv")
num_rows, num_cols = df.shape
print("Rows: ", num_rows)
print("Columns: ", num_cols)

Rows:  252175
Columns:  91


In [5]:
print("Null rows:", df.shape[0] - df.dropna().shape[0])
print("Duplicated rows:", df.duplicated().sum())
df.drop_duplicates(inplace=True)

Null rows: 0
Duplicated rows: 52


### Train Test Split

In [6]:
# Separate indices in train/val/set
# "stratify=y" makes sure to keep the classes proportions on the dataset (useful on imbalanced classes)
train, test = train_test_split(df, stratify=df["Year"], test_size=0.3, random_state=random_state)
val, test = train_test_split(test, stratify=test["Year"], test_size=(1 / 3), random_state=random_state)

X_train = train.drop(columns=["Year"])
y_train = train["Year"]

X_val = val.drop(columns=["Year"])
y_val = val["Year"]

X_test = test.drop(columns=["Year"])
y_test = test["Year"]

### Preprocessing 

In [7]:
from sklearn import preprocessing
from sklearn.covariance import OAS
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    steps=[
        ("min-max", preprocessing.MinMaxScaler()),
        ("lmax", preprocessing.Normalizer(norm="max")),
        ("lda", LinearDiscriminantAnalysis(solver="eigen", shrinkage=None, covariance_estimator=OAS())),
    ]
)

if preproc == True:
    # Fit the pipeline to the data
    pipeline.fit(X_train, y_train)

    # Transform the data using the pipeline
    X_train = pipeline.transform(X_train)
    X_test = pipeline.transform(X_test)
    X_val = pipeline.transform(X_val)

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    X_val = pd.DataFrame(X_val)

    train = pd.DataFrame(X_train)
    test = pd.DataFrame(X_test)
    val = pd.DataFrame(X_val)

    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    y_val.reset_index(drop=True, inplace=True)

    train["Year"] = y_train
    test["Year"] = y_test
    val['Year'] = y_val


## Config


In [10]:
target = ["Year"]
continous_cols = list(train.columns)[:-1]

In [11]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [12]:
data_config = DataConfig(target=target, continuous_cols=continous_cols, num_workers=0)

optimizer_config = OptimizerConfig(lr_scheduler="ReduceLROnPlateau", lr_scheduler_params={"patience": 10, "threshold": 1, "threshold_mode": "abs"},)

head_config = LinearHeadConfig(
    layers="",  # No additional layer in head, just a mapping layer to output_dim
    initialization="kaiming",
).__dict__  # Convert to dict to pass to the model config

if preproc == False:
    experiment_config = ExperimentConfig(
        project_name="TabNet",
        run_name="TabNet-raw",
        log_target="tensorboard",
    )
else:
    experiment_config = ExperimentConfig(
        project_name="TabNet",
        run_name="TabNet-preproc",
        log_target="tensorboard",
    )

## Train

In [13]:
learning_rates = [0.005]
batch_sizes = [256, 512]
virtual_batch_sizes = [128]
n_epochs = [200]
# Dimension of the prediction  layer
n_d = [32, 64]
# Dimension of the attention  layer
n_a = [16, 32]
# Number of successive steps in the network
n_steps = [3]
# Number of independent GLU layer in each GLU block
n_independent = [2, 3]
# Number of shared GLU layer in each GLU block
n_shared = [2, 3]
# Coefficient for feature reusage in the masks. A value close to 1 will make mask selection least correlated between layers. Values range from 1.0 to 2.0.
gamma = [1.5]


params = list(
    product(
        learning_rates,
        batch_sizes,
        virtual_batch_sizes,
        n_epochs,
        n_d,
        n_a,
        n_steps,
        n_independent,
        n_shared,
        gamma,
    )
)


comb = (
    len(learning_rates)
    * len(batch_sizes)
    * len(virtual_batch_sizes)
    * len(n_epochs)
    * len(n_d)
    * len(n_a)
    * len(n_steps)
    * len(n_independent)
    * len(n_shared)
    * len(gamma)
)


print("Number of combinations: ", comb)

Number of combinations:  32


### TabNet

In [14]:
best_mse = float("inf")
best_model = None
best_params = None
iter = 0
results = pd.DataFrame(
    columns=["loss", "r2", 'learning_rate', 'batch_size', 'virtual_batch_size', 'n_epochs', 'n_d', 'n_a', 'n_steps', 'n_independent', 'n_shared', 'gamma']
)


for learning_rate, batch_size, virtual_batch_size, n_epochs, n_d, n_a, n_steps, n_independent, n_shared, gamma in params:
    iter += 1
    print(f"\nIteration: {iter} of {comb}")
    trainer_config = TrainerConfig(
        batch_size=batch_size,
        max_epochs=n_epochs,
        early_stopping_patience=10,
    )

    model_config = TabNetModelConfig(
        task="regression",
        head="LinearHead",  # Linear Head
        head_config=head_config,  # Linear Head Config
        learning_rate=learning_rate,
        virtual_batch_size=virtual_batch_size,
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        n_independent=n_independent,
        n_shared=n_shared,
        gamma=gamma
    )

    tabular_model = TabularModel(
        data_config=data_config,
        model_config=model_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        experiment_config=experiment_config,
        verbose=False
    )

    tabular_model.fit(train=train, validation=val)
    tabular_model.evaluate(test)

    y_pred = tabular_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("MSE: ", mse)
    print("R2: ", r2)

    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(tabular_model)
        best_params = (learning_rate, batch_size, virtual_batch_size, n_epochs, n_d, n_a, n_steps, n_independent, n_shared, gamma)
        print("Best model updated")

    config = [mse, r2, learning_rate, batch_size, virtual_batch_size, n_epochs, n_d, n_a, n_steps, n_independent, n_shared, gamma]
    results.loc[len(results)] = config


Iteration: 1 of 32


Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Win10\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:639: Checkpoint directory C:\Users\Win10\Desktop\tabnet\saved_models exists and is not empty.


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type           ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _embedding_layer │ Identity       │      0 │
│ 1 │ _backbone        │ TabNetBackbone │ 52.7 K │
│ 2 │ _head            │ Identity       │      0 │
│ 3 │ loss             │ MSELoss        │      0 │
└───┴──────────────────┴────────────────┴────────┘

Trainable params: 52.7 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 52.7 K                                                                                               
Total estimated model params size (MB): 0

Output()

...

C:\Users\Win10\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\data_connector.p
y:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value 
of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

Output()

C:\Users\Win10\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     81.11822509765625     │
│  test_mean_squared_error  │     81.11822509765625     │
└───────────────────────────┴───────────────────────────┘

MSE:  81.11822930435045
R2:  0.263052350158423


In [15]:
results.sort_values(by="r2", ascending=False).head()

,loss,r2,learning_rate,batch_size,virtual_batch_size,n_epochs,n_d,n_a,n_steps,n_independent,n_shared,gamma
8,70.756287,0.357189,0.005,256.0,128.0,200.0,64.0,16.0,3.0,2.0,2.0,1.5
1,71.638610,0.349173,0.005,256.0,128.0,200.0,32.0,16.0,3.0,2.0,3.0,1.5
26,74.849821,0.320000,0.005,512.0,128.0,200.0,64.0,16.0,3.0,3.0,2.0,1.5
2,77.472965,0.296169,0.005,256.0,128.0,200.0,32.0,16.0,3.0,3.0,2.0,1.5
11,78.231979,0.289274,0.005,256.0,128.0,200.0,64.0,16.0,3.0,3.0,3.0,1.5


In [16]:
if preproc == True:
    results.sort_values(by="r2", ascending=False).to_csv("tabnet-preproc.csv")
else:
    results.sort_values(by="r2", ascending=False).to_csv("tabnet-raw.csv")

In [22]:
import pickle

file = open("3_" + "TB" + "_preproc_" + ".save", "wb")
pickle.dump(pipeline, file)

file = open("3_" + "TB" + "_model_" + ".save", "wb")
pickle.dump(best_model, file)
file.close()